In [1]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/shichenyang/Downloads/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-17 13:04:02,924 INFO: Initializing external client
2025-11-17 13:04:02,926 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-17 13:04:04,446 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1296535


'{"country": "denmark", "city": "copenhagen", "street": "h.c.andersens-boulevard", "aqicn_url": "https://api.waqi.info/feed/@3317", "latitude": "55.675634", "longitude": "12.56794"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,36.0,denmark,copenhagen,h.c.andersens-boulevard,2025-11-17,https://api.waqi.info/feed/@3317


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌍 Calculate Lagged Features for PM2.5 (Previous 1, 2, and 3 days)</span>

To calculate the lagged features, we need to retrieve the previous 3 days of air quality data from the feature store and compute the PM2.5 values from 1, 2, and 3 days ago.

In [7]:
# Retrieve historical air quality data to calculate lagged features
# Query the air quality feature group for the previous 3 days
query_aq = air_quality_fg.select_all()

# Get the last 3 days of data (not including today, which we haven't inserted yet)
aq_historical = query_aq.read()

# Sort by date and get the last 3 rows
aq_historical = aq_historical.sort_values('date').tail(3)
print("Historical data (last 3 days for lag features):")
print(aq_historical[['date', 'pm25']])

# Initialize lagged features as NaN
aq_today_df['pm25_lag_1d'] = float('nan')
aq_today_df['pm25_lag_2d'] = float('nan')
aq_today_df['pm25_lag_3d'] = float('nan')

# Calculate lagged features if we have sufficient historical data
if len(aq_historical) >= 1:
    # Get PM2.5 from 1 day ago (yesterday)
    aq_today_df['pm25_lag_1d'] = aq_historical.iloc[-1]['pm25']

if len(aq_historical) >= 2:
    # Get PM2.5 from 2 days ago
    aq_today_df['pm25_lag_2d'] = aq_historical.iloc[-2]['pm25']

if len(aq_historical) >= 3:
    # Get PM2.5 from 3 days ago
    aq_today_df['pm25_lag_3d'] = aq_historical.iloc[-3]['pm25']

# Convert to float32
aq_today_df['pm25_lag_1d'] = aq_today_df['pm25_lag_1d'].astype('float32')
aq_today_df['pm25_lag_2d'] = aq_today_df['pm25_lag_2d'].astype('float32')
aq_today_df['pm25_lag_3d'] = aq_today_df['pm25_lag_3d'].astype('float32')

print("\nToday's data with lagged features:")
print(aq_today_df[['date', 'pm25', 'pm25_lag_1d', 'pm25_lag_2d', 'pm25_lag_3d']])
print("\nDataFrame info:")
print(aq_today_df.info())

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.16s) 
Historical data (last 3 days for lag features):
                          date  pm25
3321 2025-11-14 00:00:00+00:00  25.0
2207 2025-11-15 00:00:00+00:00  27.0
1513 2025-11-16 00:00:00+00:00  39.0

Today's data with lagged features:
        date  pm25  pm25_lag_1d  pm25_lag_2d  pm25_lag_3d
0 2025-11-17  36.0         39.0         27.0         25.0

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   pm25         1 non-null      float32       
 1   country      1 non-null      object        
 2   city         1 non-null      object        
 3   street       1 non-null      object        
 4   date         1 non-null      datetime64[ns]
 5   url          1 non-null      object        
 6   pm25_lag_1d  1 non-null      float32       
 7   pm2

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [8]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 55.75°N 12.5°E
Elevation 8.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-17,4.60,0.0,15.815435,348.178528,copenhagen
1,2025-11-18,5.80,0.0,16.454008,259.919464,copenhagen
2,2025-11-19,3.95,0.0,3.319036,220.601212,copenhagen
3,2025-11-20,0.80,1.0,23.241617,16.189287,copenhagen
4,2025-11-21,0.15,0.0,9.659814,333.435028,copenhagen
5,2025-11-22,2.00,0.0,9.793059,216.027466,copenhagen
6,2025-11-23,2.20,0.4,22.796841,74.427399,copenhagen


In [9]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-17 13:04:11,554 INFO: 	4 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1296535/fs/1273963/fg/1730908


Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1296535/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_lag_1d",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 766993
         }
       },
       "result": {
         "observed_value": 39.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T12:04:11.000554Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_mi

In [11]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-17 13:04:25,017 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1296535/fs/1273963/fg/1721836


Uploading Dataframe: 100.00% |█| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1296535/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-17 13:05:13,348 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-17 13:05:16,516 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 13:06:36,056 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-17 13:06:39,307 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-11-17 13:06:39,773 INFO: Waiting for log aggregation to finish.
2025-11-17 13:06:39,775 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762041
         }
       },
       "result": {
         "observed_value": 3.3190360069274902,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T12:04:25.000017Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type":

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
